# Check Data Quality of the Files

checking data quality of the files
e.g. if there are redundant files or empty files, etc.

Steps:
- importing Boto because files are on S3

In [ ]:
!pip3 install s3fs boto3 mock --use-feature=2020-resolver

In [22]:
import boto3
import pandas as pd

In [16]:
def file_list(bucket_name):
    """A function that lists all files (objects) in a given bucket (argument bucket_name) and puts it in a list"""
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket_name)
    return [i.key for i in my_bucket.objects.all()]

In [17]:
file_list('dsti-ml-nlp-class')

['Data/MR_IBRAHIM_ASHAFA_STATEMENT (1).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT (2).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT (3).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT (4).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT.csv',
 'Data/MR_TAPIYA_JOHN_BANK_STATEMENT-part2.csv',
 'Data/Mr_Tochukwu_statement-part2.csv',
 'Data/mr_gani_olalekan_statement_July_9.xlsx']

In [20]:
def file_to_pandas(s3_file_uri):
    data = pd.read_csv(s3_file_uri)
    return data

In [25]:
file_to_pandas('s3://dsti-ml-nlp-class/Data/MR_IBRAHIM_ASHAFA_STATEMENT (1).csv')

ImportError: Install s3fs to access S3